# distance and direciton

Let try to opt for distance and direction with

$L1loss(y_1-y_0, y_{true})$

where $y_1=model(x_1)$

So I'm optimising for the hidden states to be the correct distance and direcioton away. It's like the margin raning loss.


links:
- [loading](https://github.com/deep-diver/LLM-As-Chatbot/blob/main/models/alpaca.py)
- [dict](https://github.com/deep-diver/LLM-As-Chatbot/blob/c79e855a492a968b54bac223e66dc9db448d6eba/model_cards.json#L143)
- [prompt_format](https://github.com/deep-diver/PingPong/blob/main/src/pingpong/alpaca.py)

In [1]:

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
plt.style.use('ggplot')

from typing import Optional, List, Dict, Union

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import Tensor
from torch import optim
from torch.utils.data import random_split, DataLoader, TensorDataset

from pathlib import Path

import transformers

import lightning.pytorch as pl
# from dataclasses import dataclass

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from sklearn.preprocessing import RobustScaler

from tqdm.auto import tqdm
import os

from loguru import logger
logger.add(os.sys.stderr, format="{time} {level} {message}", level="INFO")

transformers.__version__

'4.30.1'

# Dataset

In [2]:
from datasets import load_from_disk, concatenate_datasets
fs = [
    './.ds/model-starchat-beta_ds-amazon-polarity_format-guard-prompt_N23_3shots_088fb8'
]

# './.ds/HuggingFaceH4starchat_beta-None-N_8000-ns_3-mc_0.2-2ffc1e'
ds1 = concatenate_datasets([load_from_disk(f) for f in fs])
ds1

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:7                                                                                    │
│                                                                                                  │
│   4 ]                                                                                            │
│   5                                                                                              │
│   6 # './.ds/HuggingFaceH4starchat_beta-None-N_8000-ns_3-mc_0.2-2ffc1e'                          │
│ ❱ 7 ds1 = concatenate_datasets([load_from_disk(f) for f in fs])                                  │
│   8 ds1                                                                                          │
│   9                                                                                              │
│                                                                                                  │
│ in <listcomp>:7                                                                                  │
│                                                                                                  │
│   4 ]                                                                                            │
│   5                                                                                              │
│   6 # './.ds/HuggingFaceH4starchat_beta-None-N_8000-ns_3-mc_0.2-2ffc1e'                          │
│ ❱ 7 ds1 = concatenate_datasets([load_from_disk(f) for f in fs])                                  │
│   8 ds1                                                                                          │
│   9                                                                                              │
│                                                                                                  │
│ /home/ubuntu/mambaforge/envs/dlk2/lib/python3.9/site-packages/datasets/load.py:1886 in           │
│ load_from_disk                                                                                   │
│                                                                                                  │
│   1883 │   │   path_join = os.path.join                                                          │
│   1884 │                                                                                         │
│   1885 │   if not fs.exists(dest_dataset_path):                                                  │
│ ❱ 1886 │   │   raise FileNotFoundError(f"Directory {dataset_path} not found")                    │
│   1887 │   if fs.isfile(path_join(dest_dataset_path, config.DATASET_INFO_FILENAME)) and fs.isfi  │
│   1888 │   │   path_join(dest_dataset_path, config.DATASET_STATE_JSON_FILENAME)                  │
│   1889 │   ):                                                                                    │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
FileNotFoundError: Directory ./.ds/model-starchat-beta_ds-amazon-polarity_format-guard-prompt_N23_3shots_088fb8 not
found

In [3]:
from src.datasets.load import ds2df

## Filter

In [4]:
# lets select only the ones where
df = ds2df(ds1)
df

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:2                                                                                    │
│                                                                                                  │
│   1 # lets select only the ones where                                                            │
│ ❱ 2 df = ds2df(ds1)                                                                              │
│   3 df                                                                                           │
│   4                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'ds1' is not defined

In [5]:


# just select the question where the model knows the answer. 
d = df.query('version=="truth"').set_index("index")
# these are the ones where it got it right when asked to tell the truth
known_indices = d[d.llm_ans==d.true_answer].index

# convert to row numbers, and use datasets to select
known_rows = df['index'].isin(known_indices)
known_rows_i = df[known_rows].index

# also restrict it to significant permutations. That is monte carlo dropout pairs, where the answer changes by more than X%
m = np.abs(df.ans1-df.ans2)>0.10
significant_rows = m[m].index

allowed_rows_i = set(known_rows_i).intersection(significant_rows)
ds = ds1.select(allowed_rows_i)
ds

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:2                                                                                    │
│                                                                                                  │
│    1 # just select the question where the model knows the answer.                                │
│ ❱  2 d = df.query('version=="truth"').set_index("index")                                         │
│    3 # these are the ones where it got it right when asked to tell the truth                     │
│    4 known_indices = d[d.llm_ans==d.true_answer].index                                           │
│    5                                                                                             │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'df' is not defined

## Transform: Normalize by activation

In [6]:
# N = 1000
# small_ds = ds.select(range(N))
# b = N
# hs1 = small_ds['hs1'].reshape((b, -1))

# scaler = RobustScaler()
# hs2 = scaler.fit_transform(hs1)

# def normalize_hs(hs1, hs2):
#     b = len(hs1)
#     hs1 = scaler.transform(hs1.reshape((b, -1)))
#     hs2 = scaler.transform(hs2.reshape((b, -1)))
#     return {'hs1':hs1, 'hs2': hs2}

# # # Plot
# # plt.hist(hs1.flatten(), bins=155, range=[-5, 5], label='before', histtype='step')
# # plt.hist(hs2.flatten(), bins=155, range=[-5, 5], label='after', histtype='step')
# # plt.legend()
# # plt.show()

# # # Test
# # small_dataset = ds.select(range(4))
# # small_dataset.map(normalize_hs, batched=True, batch_size=2, input_columns=['hs1', 'hs2'])

# # run
# ds = ds.map(normalize_hs, batched=True, input_columns=['hs1', 'hs2'])
# ds

## Lightning DataModule

In [7]:
df = ds2df(ds)
df.head(4)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 df = ds2df(ds)                                                                               │
│   2 df.head(4)                                                                                   │
│   3                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'ds' is not defined

What are we detecting? If the right example of the pair is more deceptive.

Now it's only deceptive if
- it was asked to lie
- it knows the truth
- it gave the wrong answer (around 10% of the time)( it's hard to get these models to lie by encouragement rather than instruction)


In [8]:
from src.helpers import switch2bool, bool2switch
from src.datasets.dm import imdbHSDataModule

In [9]:
batch_size = 128
# test and cache
dm = imdbHSDataModule(ds, batch_size=batch_size)
dm.setup('train')

dl_val = dm.val_dataloader()
dl_train = dm.train_dataloader()


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:3                                                                                    │
│                                                                                                  │
│   1 batch_size = 128                                                                             │
│   2 # test and cache                                                                             │
│ ❱ 3 dm = imdbHSDataModule(ds, batch_size=batch_size)                                             │
│   4 dm.setup('train')                                                                            │
│   5                                                                                              │
│   6 dl_val = dm.val_dataloader()                                                                 │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'ds' is not defined

In [10]:
b = next(iter(dl_train))
x0, x1, y = b
x0.shape

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 b = next(iter(dl_train))                                                                     │
│   2 x0, x1, y = b                                                                                │
│   3 x0.shape                                                                                     │
│   4                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'dl_train' is not defined

# Data prep

We do two inferences on the same inputs. Since we have dropout enabled, even during inference, we get two slightly different hidden states `hs1` and `hs2`, and two slightly different probabilities for our yes and no output tokens `p1` `p2`. We also have the true answer `t`

So there are a few ways we can set up the problem. 

We can vary x:
- `model(hs1)-model(hs2)=y`
- `model(hs1-hs2)==y`

And we can try differen't y's:
- direction with a ranked loss. This could be unsupervised.
- magnitude with a regression loss
- vector (direction and magnitude) with a regression loss

# QC: Linear supervised probes


Let's verify that the model's representations are good

Before trying CCS, let's make sure there exists a direction that classifies examples as true vs false with high accuracy; if supervised logistic regression accuracy is bad, there's no hope of unsupervised CCS doing well.

Note that because logistic regression is supervised we expect it to do better but to have worse generalisation that equivilent unsupervised methods. However in this case CSS is using a deeper model so it is more complicated.


## Try a classification of direction to truth

In [11]:
# dm.y

In [12]:
# n = len(df)

# # Define X and y
# X = dm.hs1-dm.hs2
# y = dm.y>0

# # split
# n = len(y)
# max_rows = 1000
# print('split size', n//2)
# X_train, X_test = X[:n//2], X[n//2:]
# y_train, y_test = y[:n//2], y[n//2:]
# X_train = X_train[:max_rows]
# y_train = y_train[:max_rows]
# X_test = X_test[:max_rows]
# y_test = y_test[:max_rows]

# # scale
# scaler = RobustScaler()
# scaler.fit(X_train)
# X_train2 = scaler.transform(X_train)
# X_test2 = scaler.transform(X_test)
# print('lr')

# lr = LogisticRegression(class_weight="balanced", penalty="l2", max_iter=380)
# lr.fit(X_train2, y_train>0)

In [13]:
# print("Logistic cls acc: {:2.2%} [TRAIN]".format(lr.score(X_train2, y_train>0)))
# print("Logistic cls acc: {:2.2%} [TEST]".format(lr.score(X_test2, y_test>0)))

# m = df['lie'][n//2:][:max_rows]
# y_test_pred = lr.predict(X_test2)
# acc_w_lie = ((y_test_pred[m]>0)==(y_test[m]>0)).mean()
# acc_wo_lie = ((y_test_pred[~m]>0)==(y_test[~m]>0)).mean()
# print(f'test acc w lie {acc_w_lie:2.2%}')
# print(f'test acc wo lie {acc_wo_lie:2.2%}')

In [14]:
# primary_baseline = roc_auc_score(y_test>0, y_test_pred)
# primary_baseline

# LightningModel

In [15]:
from src.probs.conv import PLConvProbe

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 from src.probs.conv import PLConvProbe                                                       │
│   2                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ModuleNotFoundError: No module named 'src.probs'

# Run

In [16]:
# quiet please
torch.set_float32_matmul_precision('medium')

import warnings
warnings.filterwarnings("ignore", ".*does not have many workers.*")
warnings.filterwarnings("ignore", ".*F-score.*")

## Prep dataloader/set

In [17]:
dl_train = dm.train_dataloader()
dl_val = dm.val_dataloader()
b = next(iter(dl_train))
# b

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 dl_train = dm.train_dataloader()                                                             │
│   2 dl_val = dm.val_dataloader()                                                                 │
│   3 b = next(iter(dl_train))                                                                     │
│   4 # b                                                                                          │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'dm' is not defined

In [18]:
# init the model
max_epochs = 42
c_in = b[0].shape[1]
print(b[0].shape)
net = PLConvProbe(c_in=c_in, total_steps=max_epochs*len(dl_train), depth=6, hs=42*2, lr=3e-3, 
        #   weight_decay=1e-4, 
        #   dropout=0.1,
          )
net

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:3                                                                                    │
│                                                                                                  │
│    1 # init the model                                                                            │
│    2 max_epochs = 42                                                                             │
│ ❱  3 c_in = b[0].shape[1]                                                                        │
│    4 print(b[0].shape)                                                                           │
│    5 net = PLConvProbe(c_in=c_in, total_steps=max_epochs*len(dl_train), depth=6, hs=42*2, lr=    │
│    6 │   │   #   weight_decay=1e-4,                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'b' is not defined

In [19]:
# DEBUG
with torch.no_grad():
    b = next(iter(dl_train))
    b2 = [bb.to(net.device) for bb in b]
    y = net(b2[0])
y.shape, b[2].shape

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:3                                                                                    │
│                                                                                                  │
│   1 # DEBUG                                                                                      │
│   2 with torch.no_grad():                                                                        │
│ ❱ 3 │   b = next(iter(dl_train))                                                                 │
│   4 │   b2 = [bb.to(net.device) for bb in b]                                                     │
│   5 │   y = net(b2[0])                                                                           │
│   6 y.shape, b[2].shape                                                                          │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'dl_train' is not defined

In [20]:
# # DEBUG
# trainer = pl.Trainer(fast_dev_run=2)
# trainer.fit(model=net, train_dataloaders=dl_train)

In [21]:
trainer = pl.Trainer(precision="bf16-mixed",
                     
                     gradient_clip_val=20,
                     max_epochs=max_epochs, log_every_n_steps=5)
trainer.fit(model=net, train_dataloaders=dl_train, val_dataloaders=dl_val)

Using bfloat16 Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/ubuntu/mambaforge/envs/dlk2/lib/python3.9/site-packages/lightning/pytorch/trainer/connectors/logger_connector/logger_connector.py:67: UserWarning: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `lightning.pytorch` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
  warning_cache.warn(


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:5                                                                                    │
│                                                                                                  │
│   2 │   │   │   │   │                                                                            │
│   3 │   │   │   │   │    gradient_clip_val=20,                                                   │
│   4 │   │   │   │   │    max_epochs=max_epochs, log_every_n_steps=5)                             │
│ ❱ 5 trainer.fit(model=net, train_dataloaders=dl_train, val_dataloaders=dl_val)                   │
│   6                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'net' is not defined

# Read hist

In [22]:
from src.helpers.lightning import read_metrics_csv

df_hist = read_metrics_csv(trainer.logger.experiment.metrics_file_path).ffill().bfill()
df_hist

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:3                                                                                    │
│                                                                                                  │
│   1 from src.helpers.lightning import read_metrics_csv                                           │
│   2                                                                                              │
│ ❱ 3 df_hist = read_metrics_csv(trainer.logger.experiment.metrics_file_path).ffill().bfill()      │
│   4 df_hist                                                                                      │
│   5                                                                                              │
│                                                                                                  │
│ /home/ubuntu/Documents/mjc/elk/discovering_latent_knowledge/src/helpers/lightning.py:6 in        │
│ read_metrics_csv                                                                                 │
│                                                                                                  │
│    3 import pandas as pd                                                                         │
│    4                                                                                             │
│    5 def read_metrics_csv(metrics_file_path):                                                    │
│ ❱  6 │   df_hist = pd.read_csv(metrics_file_path)                                                │
│    7 │   df_hist["epoch"] = df_hist["epoch"].ffill()                                             │
│    8 │   df_histe = df_hist.set_index("epoch").groupby("epoch").mean()                           │
│    9 │   return df_histe                                                                         │
│                                                                                                  │
│ /home/ubuntu/mambaforge/envs/dlk2/lib/python3.9/site-packages/pandas/io/parsers/readers.py:912   │
│ in read_csv                                                                                      │
│                                                                                                  │
│    909 │   )                                                                                     │
│    910 │   kwds.update(kwds_defaults)                                                            │
│    911 │                                                                                         │
│ ❱  912 │   return _read(filepath_or_buffer, kwds)                                                │
│    913                                                                                           │
│    914                                                                                           │
│    915 # iterator=True -> TextFileReader                                                         │
│                                                                                                  │
│ /home/ubuntu/mambaforge/envs/dlk2/lib/python3.9/site-packages/pandas/io/parsers/readers.py:577   │
│ in _read                                                                                         │
│                                                                                                  │
│    574 │   _validate_names(kwds.get("names", None))                                              │
│    575 │                                                                                         │
│    576 │   # Create the parser.                                                                  │
│ ❱  577 │   parser = TextFileReader(filepath_or_buffer, **kwds)                                   │
│    578 │                                                                                         │
│    579 │   if chunksize or iterator:                       

In [23]:
for key in ['loss']:
    df_hist[[c for c in df_hist.columns if key in c]].plot(logy=True)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:2                                                                                    │
│                                                                                                  │
│   1 for key in ['loss']:                                                                         │
│ ❱ 2 │   df_hist[[c for c in df_hist.columns if key in c]].plot(logy=True)                        │
│   3                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'df_hist' is not defined

In [24]:
for key in ['acc', 'auroc']:
    df_hist[[c for c in df_hist.columns if key in c]].plot()

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:2                                                                                    │
│                                                                                                  │
│   1 for key in ['acc', 'auroc']:                                                                 │
│ ❱ 2 │   df_hist[[c for c in df_hist.columns if key in c]].plot()                                 │
│   3                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'df_hist' is not defined

## Predict

In [25]:
dl_test = dm.test_dataloader()
rs = trainer.test(net, dataloaders=[dl_train, dl_val, dl_test])
rs

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 dl_test = dm.test_dataloader()                                                               │
│   2 rs = trainer.test(net, dataloaders=[dl_train, dl_val, dl_test])                              │
│   3 rs                                                                                           │
│   4                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'dm' is not defined

In [26]:
dl_test = dm.test_dataloader()
r = trainer.predict(net, dataloaders=dl_test)
y_test_pred = np.concatenate(r)
y_test_pred.shape

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 dl_test = dm.test_dataloader()                                                               │
│   2 r = trainer.predict(net, dataloaders=dl_test)                                                │
│   3 y_test_pred = np.concatenate(r)                                                              │
│   4 y_test_pred.shape                                                                            │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'dm' is not defined

In [27]:
# Make a prediction dataframe with everything in it
df_test = dm.df.iloc[dm.test_split:].copy()
df_test['probe_pred'] = y_test_pred>0
y_test_pred_bool = np.clip(switch2bool(y_test_pred), 0 ,1)
df_test['probe_prob'] = y_test_pred_bool
df_test['llm_prob'] = (df_test['ans1']+df_test['ans2'])/2
df_test['llm_ans'] = df_test['llm_prob']>0.5
df_test['conf'] = (df_test['ans1']-df_test['ans2']).abs()
df_test['y'] = df_test['y']>0

y_true = dl_test.dataset.tensors[2].numpy()
assert ((df_test['y'].values>0.5)==(y_true>0)).all(), 'check it all lines up'

df_test

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:2                                                                                    │
│                                                                                                  │
│    1 # Make a prediction dataframe with everything in it                                         │
│ ❱  2 df_test = dm.df.iloc[dm.test_split:].copy()                                                 │
│    3 df_test['probe_pred'] = y_test_pred>0                                                       │
│    4 y_test_pred_bool = np.clip(switch2bool(y_test_pred), 0 ,1)                                  │
│    5 df_test['probe_prob'] = y_test_pred_bool                                                    │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'dm' is not defined

In [28]:
def get_acc_subset(df, query):
    df_s = df.query(query)
    acc = (df_s['probe_pred']==df_s['y']).mean()
    print(f"acc={acc:2.2%} [{query}]")
    return acc
    
print('probe results on subsets of the data')
get_acc_subset(df_test, 'lie==True') # it was ph told to lie
get_acc_subset(df_test, 'lie==False') # it was told not to lie
get_acc_subset(df_test, 'llm_ans==true_answer') # the llm gave the true ans
get_acc_subset(df_test, 'llm_ans==desired_answer') # the llm gave the desired ans
get_acc_subset(df_test, 'lie==True & llm_ans==desired_answer') # it was told to lie, and it did lie
get_acc_subset(df_test, 'lie==True & llm_ans!=desired_answer')

probe results on subsets of the data


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:8                                                                                    │
│                                                                                                  │
│    5 │   return acc                                                                              │
│    6                                                                                             │
│    7 print('probe results on subsets of the data')                                               │
│ ❱  8 get_acc_subset(df_test, 'lie==True') # it was ph told to lie                                │
│    9 get_acc_subset(df_test, 'lie==False') # it was told not to lie                              │
│   10 get_acc_subset(df_test, 'llm_ans==true_answer') # the llm gave the true ans                 │
│   11 get_acc_subset(df_test, 'llm_ans==desired_answer') # the llm gave the desired ans           │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'df_test' is not defined

# RESULTS

In [29]:
roc_auc = roc_auc_score(df_test['y'], y_test_pred_bool)

# print(f"  PRIMARY BASELINE roc_auc={primary_baseline:2.2%} from linear classifier")
print(f"⭐PRIMARY METRIC⭐ roc_auc={roc_auc:2.2%} from probe")

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 roc_auc = roc_auc_score(df_test['y'], y_test_pred_bool)                                      │
│   2                                                                                              │
│   3 # print(f"  PRIMARY BASELINE roc_auc={primary_baseline:2.2%} from linear classifier")        │
│   4 print(f"⭐PRIMARY METRIC⭐ roc_auc={roc_auc:2.2%} from probe")                               │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'df_test' is not defined